In [1]:
import pandas as pd

In [2]:
# Get the embeddings
embeddings = pd.read_csv("data/OsmoticStress_with_binary_positions_and_embeddings.csv", index_col=0)
embeddings = embeddings.drop(columns=["full_sequence"])

# Get the dataset
df = pd.read_csv('data/OsmoticStress_with_binary_positions.csv')
df.drop(columns=['full_sequence', 'Peptide_sequence'], inplace=True)
df.set_index('Uniprot_ID', inplace=True)

# Merge the two and get the final dataset
dataset = df.merge(embeddings, left_index=True, right_index=True)

In [12]:
dataset.head()

,Log2FC(LiP_norm),Binary_Positions,full_embedding
Uniprot_ID,,,
A5Z2X5,-0.153604,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8848178386688232, 7881519943475723, 428928285..."
A5Z2X5,-1.869623,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8848178386688232, 7881519943475723, 428928285..."
D6VTK4,-0.008114,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[47565970569849014, 7273535244166851, 22035147..."
O13297,-0.374875,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[21353999618440866, 15769580379128456, 4766600..."
O13297,-0.207803,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[21353999618440866, 15769580379128456, 4766600..."


In [4]:
import re

def convert_to_list(number_string):
    cleaned_string = re.sub(r"[^\d,\s]", "", number_string)
    return list(map(int, cleaned_string.split(',')))

dataset['full_embedding'] = dataset['full_embedding'].apply(convert_to_list)
dataset['Binary_Positions'] = dataset['Binary_Positions'].apply(convert_to_list)

In [10]:
import torch
import torch.nn as nn
import numpy as np

embedding_array = np.array(dataset['full_embedding'].tolist(), dtype=np.float32)
full_embedding = torch.tensor(embedding_array)

position_embedding_dim = len(dataset.iloc[0, 2]) 
num_positions = 20
position_embedding = nn.Embedding(num_embeddings=num_positions, embedding_dim=position_embedding_dim)

position_indices = torch.tensor([x[0] for x in dataset['Binary_Positions']])

position_embed = position_embedding(position_indices)

combined_embeddings = position_embed + full_embedding

combined_embeddings_list = combined_embeddings.tolist()

In [11]:
# Create a new DataFrame using the original DataFrame's index
X = pd.DataFrame(combined_embeddings_list, index=dataset.index)

In [15]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
Uniprot_ID,,,,,,,,,,,,,,,,,,,,,
A5Z2X5,8.848179e+15,7.881520e+15,4.289283e+16,1.710427e+16,7.472561e+15,1.246796e+15,2.859221e+15,4.647810e+14,3.386821e+13,7.752636e+15,...,5.445937e+15,5.214862e+16,5.950918e+15,2.992804e+15,2.186712e+16,2.490647e+16,9.536843e+15,2.424401e+16,3.110352e+15,3.304118e+15
A5Z2X5,8.848179e+15,7.881520e+15,4.289283e+16,1.710427e+16,7.472561e+15,1.246796e+15,2.859221e+15,4.647810e+14,3.386821e+13,7.752636e+15,...,5.445937e+15,5.214862e+16,5.950918e+15,2.992804e+15,2.186712e+16,2.490647e+16,9.536843e+15,2.424401e+16,3.110352e+15,3.304118e+15
D6VTK4,4.756597e+16,7.273535e+15,2.203515e+14,6.608049e+14,3.326045e+15,7.376782e+15,1.223982e+16,1.203883e+16,1.552944e+16,5.348460e+15,...,5.309712e+15,6.676825e+15,7.294690e+15,5.301362e+16,2.983132e+15,3.740367e+15,1.510385e+16,1.461948e+16,4.779517e+16,6.978594e+14
O13297,2.135400e+16,1.576958e+16,4.766600e+16,1.534467e+16,2.267701e+15,5.980504e+15,3.956593e+15,9.986967e+15,2.456207e+15,3.738843e+16,...,9.076984e+15,8.298277e+15,4.864088e+15,2.892880e+15,2.022724e+15,3.994240e+15,6.644006e+15,1.113248e+16,3.134675e+16,7.254029e+14
O13297,2.135400e+16,1.576958e+16,4.766600e+16,1.534467e+16,2.267701e+15,5.980504e+15,3.956593e+15,9.986967e+15,2.456207e+15,3.738843e+16,...,9.076984e+15,8.298277e+15,4.864088e+15,2.892880e+15,2.022724e+15,3.994240e+15,6.644006e+15,1.113248e+16,3.134675e+16,7.254029e+14


In [14]:
y = dataset['Log2FC(LiP_norm)']

In [16]:
y.head()

Uniprot_ID
A5Z2X5   -0.153604
A5Z2X5   -1.869623
D6VTK4   -0.008114
O13297   -0.374875
O13297   -0.207803
Name: Log2FC(LiP_norm), dtype: float64